<a href="https://colab.research.google.com/github/Rahad31/Different-VAE-for-KL-FedDis/blob/main/BvaeOrd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
%%time
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch import nn
from typing import Dict

CPU times: user 50 µs, sys: 0 ns, total: 50 µs
Wall time: 55.6 µs


In [24]:
# Define VAE model
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim, z_dim):
        super(VAE, self).__init__()
        self.x_dim = x_dim
        self.h_dim = h_dim
        self.z_dim = z_dim

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, stride=2, padding=1),  # 16x16
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2, padding=1),  # 8x8
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),  # 4x4
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(2048, 256),
            nn.ReLU(),
            nn.Linear(256, 2*z_dim)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 2048),
            nn.ReLU(),
            nn.Unflatten(1, (128, 4, 4)),
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),  # 8x8
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1),  # 16x16
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 4, stride=2, padding=1),  # 32x32
            nn.Sigmoid()
        )

    def encode(self, x):
        h = self.encoder(x)
        mu, logvar = h[:, :self.z_dim], h[:, self.z_dim:]
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


In [25]:
def vae_train(vae: VAE, trainloader: DataLoader, epochs: int, beta: float = 1.0) -> None:
    optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)
    for epoch in range(epochs):
        for i, data in enumerate(trainloader, 0):
            inputs, _ = data
            optimizer.zero_grad()
            recon_x, mu, logvar = vae(inputs)
            loss = vae_loss(recon_x, inputs, mu, logvar, beta=beta)
            loss.backward()
            optimizer.step()



In [26]:

# Define classification model
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [27]:
# Load CIFAR10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

full_dataset = CIFAR10(root="./data", train=True, download=True, transform=transform)
test_set = CIFAR10(root="./data", train=False, download=True, transform=transform)

In [28]:
# Split dataset into training and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_set, val_set = torch.utils.data.random_split(full_dataset, [train_size, val_size])


In [8]:
# Create training and validation loaders
trainloader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2)
valloader = DataLoader(val_set, batch_size=128, shuffle=False, num_workers=2)
testloader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=2)


In [29]:
def vae_loss(recon_x, x, mu, logvar, beta=1):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + beta * KLD


In [30]:
# Define training procedure for classification model
def train(net: nn.Module, trainloader: DataLoader, valloader: DataLoader, epochs: int) -> None:
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

    best_acc = 0.0

    for epoch in range(epochs):
        # Training loop
        net.train()
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validation loop
        net.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in valloader:
                images, labels = data
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = 100 * correct / total
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(net.state_dict(), 'best_model.pth')

        print(f"Epoch [{epoch+1}/{epochs}], Training Loss: {running_loss / (i+1):.3f}, Validation Accuracy: {val_acc:.2f}%")

In [31]:
# Define evaluation procedure
def evaluate(net: nn.Module, testloader: DataLoader) -> float:
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [32]:
def initialize_clients(trainset, transform, batch_size, num_clients):
    clients = {}
    for i in range(num_clients):
        client_trainset = torch.utils.data.Subset(trainset, range(i * len(trainset) // num_clients, (i + 1) * len(trainset) // num_clients))
        client_trainloader = torch.utils.data.DataLoader(client_trainset, batch_size=batch_size, shuffle=True)
        clients[f"client_{i}"] = client_trainloader
    return clients

In [33]:
def get_distribution_info(vae: VAE) -> Dict:
    # Implement the logic to extract distribution information from the VAE
    # This can involve computing statistics, parameters, or any other relevant information
    # that can be used to generate augmented data

    # Example implementation:
    distribution_info = {
        "mean": vae.encoder[-1].bias.data.cpu().numpy(),
        "std": torch.exp(0.5 * vae.encoder[-1].weight.data).cpu().numpy()
    }

    return distribution_info

In [34]:
def send_distribution_info(distribution_info: Dict) -> None:
    # Implement the logic to send the distribution information to the global server
    # This can involve using a network protocol, a message queue, or any other communication mechanism
    # to send the information

    # Example implementation:
    # Send the distribution information to the global server using a network protocol
    # For example, you can use the `socket` module to send the information over a network
    # or use a message queue like `RabbitMQ` to send the information
    pass


In [35]:
# Define logic to generate augmented data using Ordinary Normal distribution
def generate_augmented_data(vae: VAE, distribution_info: Dict) -> torch.Tensor:
    # Generate augmented data using Ordinary Normal distribution
    mean = distribution_info["mean"]
    std = distribution_info["std"]
    augmented_data = torch.randn(64, vae.z_dim) * std + mean
    return augmented_data

In [36]:
def federated_train(net: nn.Module, vae: VAE, trainloaders: Dict[str, DataLoader], trainloader: DataLoader, valloader: DataLoader, epochs: int) -> None:
    for epoch in range(epochs):
        for client_id, client_trainloader in trainloaders.items():
            # Train VAE on client data
            vae_train(vae, client_trainloader, epochs=10, beta=1)

            # Share distribution information with global server
            distribution_info = get_distribution_info(vae)
            send_distribution_info(distribution_info)

            # Receive distribution information from other clients
            other_distribution_info = receive_distribution_info()

            # Generate augmented data using received distribution information
            augmented_data = generate_augmented_data(vae, other_distribution_info)

            # Train classification model using local, augmented, and validation data
            train(net, client_trainloader, valloader, epochs=10)

            # Send model updates to global server
            send_model_update(client_id, net.state_dict())

In [37]:
# Define logic to receive distribution information from global server
def receive_distribution_info() -> Dict:
    # Receive distribution information logic
    distribution_info = {
        "mean": np.zeros(20),  # Adjust the size based on your latent space dimension
        "std": np.ones(20)
    }
    return distribution_info

In [38]:
def send_model_update(client_id: str, model_update: Dict) -> None:
    # Implement the logic to send the model update to the global server
    # This can involve using a network protocol, a message queue, or any other communication mechanism
    # to send the model update

    # Example implementation:
    # Send the model update to the global server using a network protocol
    # For example, you can use the `socket` module to send the information over a network
    # or use a message queue like `RabbitMQ` to send the information
    pass

***-------For 250 Epochs B=4.0-------***

---



In [ ]:
%%time

# Define global server procedure
def global_server() -> None:
    net = Net()
    x_dim = 3 * 32 * 32  # CIFAR-10 input size
    h_dim = 400
    z_dim = 20
    vae = VAE(x_dim, h_dim, z_dim)  # Initialize VAE object with required arguments

    # Initialize clients
    num_clients = 5  # Define the number of clients
    clients = initialize_clients(train_set, transform, batch_size=128, num_clients=num_clients)

    # Train model using FedDIS
    federated_train(net, vae, clients, trainloader, valloader, epochs=5)

    # Evaluate final model
    test_accuracy = evaluate(net, testloader)
    print(f"Test Accuracy: {test_accuracy:.2f}%")

if __name__ == "__main__":
    global_server()


/tmp/ipython-input-24-3419885715.py:6: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  augmented_data = torch.randn(64, vae.z_dim) * std + mean


Epoch [1/10], Training Loss: 2.305, Validation Accuracy: 10.51%
Epoch [2/10], Training Loss: 2.304, Validation Accuracy: 11.08%
Epoch [3/10], Training Loss: 2.303, Validation Accuracy: 11.00%
Epoch [4/10], Training Loss: 2.302, Validation Accuracy: 10.34%
Epoch [5/10], Training Loss: 2.302, Validation Accuracy: 10.15%
Epoch [6/10], Training Loss: 2.301, Validation Accuracy: 10.08%
Epoch [7/10], Training Loss: 2.300, Validation Accuracy: 10.08%
Epoch [8/10], Training Loss: 2.299, Validation Accuracy: 10.08%
Epoch [9/10], Training Loss: 2.299, Validation Accuracy: 10.08%
Epoch [10/10], Training Loss: 2.297, Validation Accuracy: 10.09%
Epoch [1/10], Training Loss: 2.297, Validation Accuracy: 10.14%
Epoch [2/10], Training Loss: 2.296, Validation Accuracy: 10.60%
Epoch [3/10], Training Loss: 2.294, Validation Accuracy: 12.86%
Epoch [4/10], Training Loss: 2.291, Validation Accuracy: 15.67%
Epoch [5/10], Training Loss: 2.288, Validation Accuracy: 16.95%
Epoch [6/10], Training Loss: 2.283, Val

***-------For 500 Epochs B=4.0-------***

---

In [ ]:
# Define global server procedure
def global_server() -> None:
    net = Net()
    x_dim = 3 * 32 * 32  # CIFAR-10 input size
    h_dim = 400
    z_dim = 20
    vae = VAE(x_dim, h_dim, z_dim)  # Initialize VAE object with required arguments

    # Initialize clients
    num_clients = 5  # Define the number of clients
    clients = initialize_clients(train_set, transform, batch_size=128, num_clients=num_clients)

    # Train model using FedDIS
    federated_train(net, vae, clients, trainloader, valloader, epochs=10)

    # Evaluate final model
    test_accuracy, tp, fp, tn, fn, precision, recall, f1 = evaluate(net, testloader)
    print(f"Test Accuracy: {test_accuracy:.2f}%")

    print("True Positives (TP):", tp)
    print("False Positives (FP):", fp)
    print("True Negatives (TN):", tn)
    print("False Negatives (FN):", fn)
    print(":", fn+tn+tp+fp)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

if __name__ == "__main__":
    global_server()

/tmp/ipython-input-43-3419885715.py:6: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  augmented_data = torch.randn(64, vae.z_dim) * std + mean


Epoch [1/10], Training Loss: 2.306, Validation Accuracy: 10.74%
Epoch [2/10], Training Loss: 2.305, Validation Accuracy: 10.77%
Epoch [3/10], Training Loss: 2.304, Validation Accuracy: 10.82%
Epoch [4/10], Training Loss: 2.303, Validation Accuracy: 10.87%
Epoch [5/10], Training Loss: 2.303, Validation Accuracy: 11.09%
Epoch [6/10], Training Loss: 2.302, Validation Accuracy: 11.48%
Epoch [7/10], Training Loss: 2.301, Validation Accuracy: 12.17%
Epoch [8/10], Training Loss: 2.300, Validation Accuracy: 12.77%
Epoch [9/10], Training Loss: 2.299, Validation Accuracy: 13.61%
Epoch [10/10], Training Loss: 2.298, Validation Accuracy: 15.41%
Epoch [1/10], Training Loss: 2.297, Validation Accuracy: 16.92%
Epoch [2/10], Training Loss: 2.295, Validation Accuracy: 17.60%
Epoch [3/10], Training Loss: 2.292, Validation Accuracy: 17.82%
Epoch [4/10], Training Loss: 2.288, Validation Accuracy: 17.94%
Epoch [5/10], Training Loss: 2.283, Validation Accuracy: 18.24%
Epoch [6/10], Training Loss: 2.275, Val

TypeError: cannot unpack non-iterable float object

In [19]:
%%time

# Define global server procedure
def global_server() -> None:
    net = Net()
    x_dim = 3 * 32 * 32  # CIFAR-10 input size
    h_dim = 400
    z_dim = 20
    vae = VAE(x_dim, h_dim, z_dim)  # Initialize VAE object with required arguments

    # Initialize clients
    num_clients = 5  # Define the number of clients
    clients = initialize_clients(train_set, transform, batch_size=128, num_clients=num_clients)

    # Train model using FedDIS
    federated_train(net, vae, clients, trainloader, valloader, epochs=10)

    # Evaluate final model
    test_accuracy = evaluate(net, testloader)
    print(f"Test Accuracy: {test_accuracy:.2f}%")

if __name__ == "__main__":
    global_server()

/tmp/ipython-input-15-3419885715.py:6: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  augmented_data = torch.randn(64, vae.z_dim) * std + mean


Epoch [1/10], Training Loss: 2.304, Validation Accuracy: 10.38%
Epoch [2/10], Training Loss: 2.303, Validation Accuracy: 10.94%
Epoch [3/10], Training Loss: 2.302, Validation Accuracy: 11.72%
Epoch [4/10], Training Loss: 2.301, Validation Accuracy: 11.68%
Epoch [5/10], Training Loss: 2.300, Validation Accuracy: 11.60%
Epoch [6/10], Training Loss: 2.299, Validation Accuracy: 11.38%
Epoch [7/10], Training Loss: 2.297, Validation Accuracy: 11.52%
Epoch [8/10], Training Loss: 2.296, Validation Accuracy: 12.27%
Epoch [9/10], Training Loss: 2.293, Validation Accuracy: 13.94%
Epoch [10/10], Training Loss: 2.290, Validation Accuracy: 15.19%
Epoch [1/10], Training Loss: 2.288, Validation Accuracy: 15.12%
Epoch [2/10], Training Loss: 2.282, Validation Accuracy: 15.61%
Epoch [3/10], Training Loss: 2.273, Validation Accuracy: 16.64%
Epoch [4/10], Training Loss: 2.261, Validation Accuracy: 17.53%
Epoch [5/10], Training Loss: 2.243, Validation Accuracy: 19.21%
Epoch [6/10], Training Loss: 2.219, Val

***-------For 250 Epochs B=1.0-------***

---


In [39]:
%%time

# Define global server procedure
def global_server() -> None:
    net = Net()
    x_dim = 3 * 32 * 32  # CIFAR-10 input size
    h_dim = 400
    z_dim = 20
    vae = VAE(x_dim, h_dim, z_dim)  # Initialize VAE object with required arguments

    # Initialize clients
    num_clients = 5  # Define the number of clients
    clients = initialize_clients(train_set, transform, batch_size=128, num_clients=num_clients)

    # Train model using FedDIS
    federated_train(net, vae, clients, trainloader, valloader, epochs=5)

    # Evaluate final model
    test_accuracy = evaluate(net, testloader)
    print(f"Test Accuracy: {test_accuracy:.2f}%")

if __name__ == "__main__":
    global_server()


/tmp/ipython-input-35-3419885715.py:6: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  augmented_data = torch.randn(64, vae.z_dim) * std + mean


Epoch [1/10], Training Loss: 2.304, Validation Accuracy: 10.70%
Epoch [2/10], Training Loss: 2.303, Validation Accuracy: 10.85%
Epoch [3/10], Training Loss: 2.302, Validation Accuracy: 10.96%
Epoch [4/10], Training Loss: 2.301, Validation Accuracy: 10.90%
Epoch [5/10], Training Loss: 2.301, Validation Accuracy: 11.05%
Epoch [6/10], Training Loss: 2.300, Validation Accuracy: 10.86%
Epoch [7/10], Training Loss: 2.299, Validation Accuracy: 11.40%
Epoch [8/10], Training Loss: 2.298, Validation Accuracy: 11.90%
Epoch [9/10], Training Loss: 2.296, Validation Accuracy: 13.73%
Epoch [10/10], Training Loss: 2.294, Validation Accuracy: 14.81%
Epoch [1/10], Training Loss: 2.293, Validation Accuracy: 16.03%
Epoch [2/10], Training Loss: 2.290, Validation Accuracy: 16.87%
Epoch [3/10], Training Loss: 2.286, Validation Accuracy: 17.39%
Epoch [4/10], Training Loss: 2.279, Validation Accuracy: 16.82%
Epoch [5/10], Training Loss: 2.269, Validation Accuracy: 17.13%
Epoch [6/10], Training Loss: 2.252, Val

-----For 500 epoachs B=1--------


In [ ]:
%%time

# Define global server procedure
def global_server() -> None:
    net = Net()
    x_dim = 3 * 32 * 32  # CIFAR-10 input size
    h_dim = 400
    z_dim = 20
    vae = VAE(x_dim, h_dim, z_dim)  # Initialize VAE object with required arguments

    # Initialize clients
    num_clients = 5  # Define the number of clients
    clients = initialize_clients(train_set, transform, batch_size=128, num_clients=num_clients)

    # Train model using FedDIS
    federated_train(net, vae, clients, trainloader, valloader, epochs=10)

    # Evaluate final model
    test_accuracy = evaluate(net, testloader)
    print(f"Test Accuracy: {test_accuracy:.2f}%")

if __name__ == "__main__":
    global_server()

/tmp/ipython-input-35-3419885715.py:6: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  augmented_data = torch.randn(64, vae.z_dim) * std + mean


Epoch [1/10], Training Loss: 2.303, Validation Accuracy: 9.99%
Epoch [2/10], Training Loss: 2.302, Validation Accuracy: 9.99%
Epoch [3/10], Training Loss: 2.302, Validation Accuracy: 10.15%
Epoch [4/10], Training Loss: 2.301, Validation Accuracy: 10.62%
Epoch [5/10], Training Loss: 2.300, Validation Accuracy: 10.49%
Epoch [6/10], Training Loss: 2.299, Validation Accuracy: 10.43%
Epoch [7/10], Training Loss: 2.298, Validation Accuracy: 11.52%
Epoch [8/10], Training Loss: 2.296, Validation Accuracy: 13.86%
Epoch [9/10], Training Loss: 2.294, Validation Accuracy: 15.12%
Epoch [10/10], Training Loss: 2.292, Validation Accuracy: 14.71%
Epoch [1/10], Training Loss: 2.290, Validation Accuracy: 15.41%
Epoch [2/10], Training Loss: 2.286, Validation Accuracy: 16.32%
Epoch [3/10], Training Loss: 2.280, Validation Accuracy: 16.72%
Epoch [4/10], Training Loss: 2.272, Validation Accuracy: 17.02%
Epoch [5/10], Training Loss: 2.260, Validation Accuracy: 17.16%
Epoch [6/10], Training Loss: 2.244, Valid

In [ ]:
%%time
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch import nn
from typing import Dict

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define VAE model
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim, z_dim):
        super(VAE, self).__init__()
        self.x_dim = x_dim
        self.h_dim = h_dim
        self.z_dim = z_dim

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(2048, 256),
            nn.ReLU(),
            nn.Linear(256, 2*z_dim)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 2048),
            nn.ReLU(),
            nn.Unflatten(1, (128, 4, 4)),
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 4, stride=2, padding=1),
            nn.Sigmoid()
        )

    def encode(self, x):
        h = self.encoder(x)
        mu, logvar = h[:, :self.z_dim], h[:, self.z_dim:]
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


# Define β-VAE loss function
def vae_loss(recon_x, x, mu, logvar, beta=4.0):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + beta * KLD


# Define VAE training procedure
def vae_train(vae: VAE, trainloader: DataLoader, epochs: int, beta=4.0, device='cpu') -> None:
    vae.to(device)
    optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)
    vae.train()
    for epoch in range(epochs):
        for i, data in enumerate(trainloader, 0):
            inputs, _ = data
            inputs = inputs.to(device)
            optimizer.zero_grad()
            recon_x, mu, logvar = vae(inputs)
            loss = vae_loss(recon_x, inputs, mu, logvar, beta)
            loss.backward()
            optimizer.step()


# Define classification model
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Load CIFAR10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

full_dataset = CIFAR10(root="./data", train=True, download=True, transform=transform)
test_set = CIFAR10(root="./data", train=False, download=True, transform=transform)

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_set, val_set = torch.utils.data.random_split(full_dataset, [train_size, val_size])

trainloader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2)
valloader = DataLoader(val_set, batch_size=128, shuffle=False, num_workers=2)
testloader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=2)


# Training procedure for classifier
def train(net: nn.Module, trainloader: DataLoader, valloader: DataLoader, epochs: int, device='cpu') -> None:
    net.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

    best_acc = 0.0
    for epoch in range(epochs):
        net.train()
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        net.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in valloader:
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = 100 * correct / total
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(net.state_dict(), 'best_model.pth')

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / (i+1):.3f}, Val Acc: {val_acc:.2f}%")


# Evaluation procedure
def evaluate(net: nn.Module, testloader: DataLoader) -> float:
    net.to(device)
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total


# Initialize simulated clients
def initialize_clients(trainset, transform, batch_size, num_clients):
    clients = {}
    for i in range(num_clients):
        start = i * len(trainset) // num_clients
        end = (i + 1) * len(trainset) // num_clients
        client_data = torch.utils.data.Subset(trainset, list(range(start, end)))
        clients[f"client_{i}"] = DataLoader(client_data, batch_size=batch_size, shuffle=True)
    return clients


# Distribution info (using client data)
def get_distribution_info(vae: VAE, dataloader: DataLoader, device='cpu') -> Dict:
    vae.eval()
    mu_list, logvar_list = [], []
    with torch.no_grad():
        for data in dataloader:
            inputs, _ = data
            inputs = inputs.to(device)
            mu, logvar = vae.encode(inputs)
            mu_list.append(mu.cpu())
            logvar_list.append(logvar.cpu())
    mu_all = torch.cat(mu_list)
    logvar_all = torch.cat(logvar_list)
    mean = mu_all.mean(dim=0).numpy()
    std = torch.exp(0.5 * logvar_all).mean(dim=0).numpy()
    return {"mean": mean, "std": std}


# Augmentation from other clients
def generate_augmented_data(vae: VAE, distribution_info: Dict) -> torch.Tensor:
    mean = torch.tensor(distribution_info["mean"], dtype=vae.decoder[0].weight.dtype)
    std = torch.tensor(distribution_info["std"], dtype=vae.decoder[0].weight.dtype)
    z = torch.randn(64, vae.z_dim, dtype=vae.decoder[0].weight.dtype) * std + mean
    generated = vae.decode(z.to(device))
    return generated


def receive_distribution_info() -> Dict:
    return {"mean": np.zeros(20), "std": np.ones(20)}


def send_distribution_info(info: Dict): pass
def send_model_update(client_id: str, model_update: Dict): pass


def federated_train(net: nn.Module, vae: VAE, trainloaders: Dict[str, DataLoader], trainloader: DataLoader, valloader: DataLoader, epochs: int) -> None:
    for epoch in range(epochs):
        for client_id, client_loader in trainloaders.items():
            vae_train(vae, client_loader, epochs=5, beta=4.0, device=device)
            dist_info = get_distribution_info(vae, client_loader, device)
            send_distribution_info(dist_info)
            other_info = receive_distribution_info()
            aug_data = generate_augmented_data(vae, other_info)
            train(net, client_loader, valloader, epochs=1, device=device)
            send_model_update(client_id, net.state_dict())


# Global server
def global_server() -> None:
    net = Net()
    vae = VAE(x_dim=3*32*32, h_dim=400, z_dim=20)
    clients = initialize_clients(train_set, transform, batch_size=128, num_clients=5)
    federated_train(net, vae, clients, trainloader, valloader, epochs=5)
    test_acc = evaluate(net, testloader)
    print(f"Test Accuracy: {test_acc:.2f}%")


if __name__ == "__main__":
    global_server()

Epoch [1/1], Loss: 2.304, Val Acc: 9.87%
Epoch [1/1], Loss: 2.304, Val Acc: 9.74%
Epoch [1/1], Loss: 2.302, Val Acc: 9.74%
Epoch [1/1], Loss: 2.302, Val Acc: 9.76%
Epoch [1/1], Loss: 2.301, Val Acc: 9.83%
Epoch [1/1], Loss: 2.301, Val Acc: 10.04%
Epoch [1/1], Loss: 2.300, Val Acc: 9.78%
Epoch [1/1], Loss: 2.299, Val Acc: 9.81%
Epoch [1/1], Loss: 2.298, Val Acc: 10.78%
Epoch [1/1], Loss: 2.297, Val Acc: 13.31%
Epoch [1/1], Loss: 2.296, Val Acc: 15.81%
Epoch [1/1], Loss: 2.294, Val Acc: 16.77%
Epoch [1/1], Loss: 2.292, Val Acc: 17.76%
Epoch [1/1], Loss: 2.290, Val Acc: 19.16%
Epoch [1/1], Loss: 2.286, Val Acc: 19.82%
Epoch [1/1], Loss: 2.282, Val Acc: 20.03%
Epoch [1/1], Loss: 2.274, Val Acc: 20.48%
Epoch [1/1], Loss: 2.263, Val Acc: 20.99%
Epoch [1/1], Loss: 2.244, Val Acc: 21.35%
Epoch [1/1], Loss: 2.217, Val Acc: 21.51%
Epoch [1/1], Loss: 2.183, Val Acc: 22.25%
Epoch [1/1], Loss: 2.144, Val Acc: 22.99%
Epoch [1/1], Loss: 2.118, Val Acc: 24.10%
Epoch [1/1], Loss: 2.086, Val Acc: 24.16%

In [ ]:
%%time
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch import nn
from typing import Dict

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define VAE model
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim, z_dim):
        super(VAE, self).__init__()
        self.x_dim = x_dim
        self.h_dim = h_dim
        self.z_dim = z_dim

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(2048, 256),
            nn.ReLU(),
            nn.Linear(256, 2*z_dim)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 2048),
            nn.ReLU(),
            nn.Unflatten(1, (128, 4, 4)),
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 4, stride=2, padding=1),
            nn.Sigmoid()
        )

    def encode(self, x):
        h = self.encoder(x)
        mu, logvar = h[:, :self.z_dim], h[:, self.z_dim:]
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


# Define β-VAE loss function
def vae_loss(recon_x, x, mu, logvar, beta=1.0):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + beta * KLD


# Define VAE training procedure
def vae_train(vae: VAE, trainloader: DataLoader, epochs: int, beta=4.0, device='cpu') -> None:
    vae.to(device)
    optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)
    vae.train()
    for epoch in range(epochs):
        for i, data in enumerate(trainloader, 0):
            inputs, _ = data
            inputs = inputs.to(device)
            optimizer.zero_grad()
            recon_x, mu, logvar = vae(inputs)
            loss = vae_loss(recon_x, inputs, mu, logvar, beta)
            loss.backward()
            optimizer.step()


# Define classification model
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Load CIFAR10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

full_dataset = CIFAR10(root="./data", train=True, download=True, transform=transform)
test_set = CIFAR10(root="./data", train=False, download=True, transform=transform)

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_set, val_set = torch.utils.data.random_split(full_dataset, [train_size, val_size])

trainloader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2)
valloader = DataLoader(val_set, batch_size=128, shuffle=False, num_workers=2)
testloader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=2)


# Training procedure for classifier
def train(net: nn.Module, trainloader: DataLoader, valloader: DataLoader, epochs: int, device='cpu') -> None:
    net.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

    best_acc = 0.0
    for epoch in range(epochs):
        net.train()
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        net.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in valloader:
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = 100 * correct / total
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(net.state_dict(), 'best_model.pth')

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / (i+1):.3f}, Val Acc: {val_acc:.2f}%")


# Evaluation procedure
def evaluate(net: nn.Module, testloader: DataLoader) -> float:
    net.to(device)
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total


# Initialize simulated clients
def initialize_clients(trainset, transform, batch_size, num_clients):
    clients = {}
    for i in range(num_clients):
        start = i * len(trainset) // num_clients
        end = (i + 1) * len(trainset) // num_clients
        client_data = torch.utils.data.Subset(trainset, list(range(start, end)))
        clients[f"client_{i}"] = DataLoader(client_data, batch_size=batch_size, shuffle=True)
    return clients


# Distribution info (using client data)
def get_distribution_info(vae: VAE, dataloader: DataLoader, device='cpu') -> Dict:
    vae.eval()
    mu_list, logvar_list = [], []
    with torch.no_grad():
        for data in dataloader:
            inputs, _ = data
            inputs = inputs.to(device)
            mu, logvar = vae.encode(inputs)
            mu_list.append(mu.cpu())
            logvar_list.append(logvar.cpu())
    mu_all = torch.cat(mu_list)
    logvar_all = torch.cat(logvar_list)
    mean = mu_all.mean(dim=0).numpy()
    std = torch.exp(0.5 * logvar_all).mean(dim=0).numpy()
    return {"mean": mean, "std": std}


# Augmentation from other clients
def generate_augmented_data(vae: VAE, distribution_info: Dict) -> torch.Tensor:
    mean = torch.tensor(distribution_info["mean"], dtype=vae.decoder[0].weight.dtype)
    std = torch.tensor(distribution_info["std"], dtype=vae.decoder[0].weight.dtype)
    z = torch.randn(64, vae.z_dim, dtype=vae.decoder[0].weight.dtype) * std + mean
    generated = vae.decode(z.to(device))
    return generated


def receive_distribution_info() -> Dict:
    return {"mean": np.zeros(20), "std": np.ones(20)}


def send_distribution_info(info: Dict): pass
def send_model_update(client_id: str, model_update: Dict): pass


def federated_train(net: nn.Module, vae: VAE, trainloaders: Dict[str, DataLoader], trainloader: DataLoader, valloader: DataLoader, epochs: int) -> None:
    for epoch in range(epochs):
        for client_id, client_loader in trainloaders.items():
            vae_train(vae, client_loader, epochs=5, beta=4.0, device=device)
            dist_info = get_distribution_info(vae, client_loader, device)
            send_distribution_info(dist_info)
            other_info = receive_distribution_info()
            aug_data = generate_augmented_data(vae, other_info)
            train(net, client_loader, valloader, epochs=1, device=device)
            send_model_update(client_id, net.state_dict())


# Global server
def global_server() -> None:
    net = Net()
    vae = VAE(x_dim=3*32*32, h_dim=400, z_dim=20)
    clients = initialize_clients(train_set, transform, batch_size=128, num_clients=5)
    federated_train(net, vae, clients, trainloader, valloader, epochs=5)
    test_acc = evaluate(net, testloader)
    print(f"Test Accuracy: {test_acc:.2f}%")


if __name__ == "__main__":
    global_server()

Epoch [1/1], Loss: 2.302, Val Acc: 9.48%
Epoch [1/1], Loss: 2.301, Val Acc: 9.52%
Epoch [1/1], Loss: 2.300, Val Acc: 9.25%
Epoch [1/1], Loss: 2.298, Val Acc: 9.65%
Epoch [1/1], Loss: 2.297, Val Acc: 11.89%
Epoch [1/1], Loss: 2.295, Val Acc: 12.20%
Epoch [1/1], Loss: 2.293, Val Acc: 13.87%
Epoch [1/1], Loss: 2.290, Val Acc: 16.91%
Epoch [1/1], Loss: 2.286, Val Acc: 17.25%
Epoch [1/1], Loss: 2.281, Val Acc: 17.45%
Epoch [1/1], Loss: 2.273, Val Acc: 17.81%
Epoch [1/1], Loss: 2.264, Val Acc: 18.73%
Epoch [1/1], Loss: 2.252, Val Acc: 20.64%
Epoch [1/1], Loss: 2.234, Val Acc: 20.92%
Epoch [1/1], Loss: 2.205, Val Acc: 21.57%
Epoch [1/1], Loss: 2.176, Val Acc: 25.18%
Epoch [1/1], Loss: 2.141, Val Acc: 24.91%
Epoch [1/1], Loss: 2.107, Val Acc: 25.89%
Epoch [1/1], Loss: 2.087, Val Acc: 25.44%
Epoch [1/1], Loss: 2.049, Val Acc: 26.19%
Epoch [1/1], Loss: 2.042, Val Acc: 27.60%
Epoch [1/1], Loss: 2.030, Val Acc: 28.19%
Epoch [1/1], Loss: 2.010, Val Acc: 28.29%
Epoch [1/1], Loss: 1.998, Val Acc: 28.

In [20]:
import re

# Your provided text
log = """
Epoch [1/10], Training Loss: 2.304, Validation Accuracy: 10.38%
Epoch [2/10], Training Loss: 2.303, Validation Accuracy: 10.94%
Epoch [3/10], Training Loss: 2.302, Validation Accuracy: 11.72%
Epoch [4/10], Training Loss: 2.301, Validation Accuracy: 11.68%
Epoch [5/10], Training Loss: 2.300, Validation Accuracy: 11.60%
Epoch [6/10], Training Loss: 2.299, Validation Accuracy: 11.38%
Epoch [7/10], Training Loss: 2.297, Validation Accuracy: 11.52%
Epoch [8/10], Training Loss: 2.296, Validation Accuracy: 12.27%
Epoch [9/10], Training Loss: 2.293, Validation Accuracy: 13.94%
Epoch [10/10], Training Loss: 2.290, Validation Accuracy: 15.19%
Epoch [1/10], Training Loss: 2.288, Validation Accuracy: 15.12%
Epoch [2/10], Training Loss: 2.282, Validation Accuracy: 15.61%
Epoch [3/10], Training Loss: 2.273, Validation Accuracy: 16.64%
Epoch [4/10], Training Loss: 2.261, Validation Accuracy: 17.53%
Epoch [5/10], Training Loss: 2.243, Validation Accuracy: 19.21%
Epoch [6/10], Training Loss: 2.219, Validation Accuracy: 21.53%
Epoch [7/10], Training Loss: 2.186, Validation Accuracy: 23.01%
Epoch [8/10], Training Loss: 2.149, Validation Accuracy: 23.34%
Epoch [9/10], Training Loss: 2.110, Validation Accuracy: 24.25%
Epoch [10/10], Training Loss: 2.080, Validation Accuracy: 25.36%
Epoch [1/10], Training Loss: 2.050, Validation Accuracy: 26.30%
Epoch [2/10], Training Loss: 2.031, Validation Accuracy: 26.39%
Epoch [3/10], Training Loss: 2.014, Validation Accuracy: 27.61%
Epoch [4/10], Training Loss: 1.994, Validation Accuracy: 28.12%
Epoch [5/10], Training Loss: 1.978, Validation Accuracy: 28.83%
Epoch [6/10], Training Loss: 1.957, Validation Accuracy: 29.58%
Epoch [7/10], Training Loss: 1.938, Validation Accuracy: 30.28%
Epoch [8/10], Training Loss: 1.921, Validation Accuracy: 30.92%
Epoch [9/10], Training Loss: 1.905, Validation Accuracy: 31.31%
Epoch [10/10], Training Loss: 1.887, Validation Accuracy: 32.00%
Epoch [1/10], Training Loss: 1.898, Validation Accuracy: 32.39%
Epoch [2/10], Training Loss: 1.878, Validation Accuracy: 33.49%
Epoch [3/10], Training Loss: 1.854, Validation Accuracy: 34.76%
Epoch [4/10], Training Loss: 1.830, Validation Accuracy: 35.22%
Epoch [5/10], Training Loss: 1.805, Validation Accuracy: 35.60%
Epoch [6/10], Training Loss: 1.790, Validation Accuracy: 33.84%
Epoch [7/10], Training Loss: 1.777, Validation Accuracy: 35.93%
Epoch [8/10], Training Loss: 1.752, Validation Accuracy: 36.39%
Epoch [9/10], Training Loss: 1.732, Validation Accuracy: 37.76%
Epoch [10/10], Training Loss: 1.718, Validation Accuracy: 37.77%
Epoch [1/10], Training Loss: 1.717, Validation Accuracy: 38.29%
Epoch [2/10], Training Loss: 1.698, Validation Accuracy: 38.36%
Epoch [3/10], Training Loss: 1.682, Validation Accuracy: 38.67%
Epoch [4/10], Training Loss: 1.667, Validation Accuracy: 39.58%
Epoch [5/10], Training Loss: 1.657, Validation Accuracy: 39.57%
Epoch [6/10], Training Loss: 1.649, Validation Accuracy: 40.46%
Epoch [7/10], Training Loss: 1.644, Validation Accuracy: 40.25%
Epoch [8/10], Training Loss: 1.627, Validation Accuracy: 40.88%
Epoch [9/10], Training Loss: 1.612, Validation Accuracy: 41.43%
Epoch [10/10], Training Loss: 1.601, Validation Accuracy: 40.36%
Epoch [1/10], Training Loss: 1.616, Validation Accuracy: 41.45%
Epoch [2/10], Training Loss: 1.601, Validation Accuracy: 41.22%
Epoch [3/10], Training Loss: 1.595, Validation Accuracy: 41.71%
Epoch [4/10], Training Loss: 1.584, Validation Accuracy: 42.18%
Epoch [5/10], Training Loss: 1.573, Validation Accuracy: 41.95%
Epoch [6/10], Training Loss: 1.561, Validation Accuracy: 43.21%
Epoch [7/10], Training Loss: 1.549, Validation Accuracy: 42.77%
Epoch [8/10], Training Loss: 1.543, Validation Accuracy: 42.95%
Epoch [9/10], Training Loss: 1.532, Validation Accuracy: 42.75%
Epoch [10/10], Training Loss: 1.524, Validation Accuracy: 43.66%
Epoch [1/10], Training Loss: 1.569, Validation Accuracy: 43.44%
Epoch [2/10], Training Loss: 1.556, Validation Accuracy: 44.21%
Epoch [3/10], Training Loss: 1.546, Validation Accuracy: 44.34%
Epoch [4/10], Training Loss: 1.531, Validation Accuracy: 44.06%
Epoch [5/10], Training Loss: 1.531, Validation Accuracy: 44.80%
Epoch [6/10], Training Loss: 1.515, Validation Accuracy: 45.22%
Epoch [7/10], Training Loss: 1.504, Validation Accuracy: 45.04%
Epoch [8/10], Training Loss: 1.499, Validation Accuracy: 45.50%
Epoch [9/10], Training Loss: 1.495, Validation Accuracy: 44.99%
Epoch [10/10], Training Loss: 1.482, Validation Accuracy: 45.80%
Epoch [1/10], Training Loss: 1.501, Validation Accuracy: 46.15%
Epoch [2/10], Training Loss: 1.483, Validation Accuracy: 46.18%
Epoch [3/10], Training Loss: 1.482, Validation Accuracy: 45.19%
Epoch [4/10], Training Loss: 1.473, Validation Accuracy: 46.26%
Epoch [5/10], Training Loss: 1.457, Validation Accuracy: 46.43%
Epoch [6/10], Training Loss: 1.447, Validation Accuracy: 47.02%
Epoch [7/10], Training Loss: 1.442, Validation Accuracy: 47.07%
Epoch [8/10], Training Loss: 1.432, Validation Accuracy: 47.26%
Epoch [9/10], Training Loss: 1.429, Validation Accuracy: 46.92%
Epoch [10/10], Training Loss: 1.423, Validation Accuracy: 47.22%
Epoch [1/10], Training Loss: 1.468, Validation Accuracy: 46.90%
Epoch [2/10], Training Loss: 1.447, Validation Accuracy: 47.72%
Epoch [3/10], Training Loss: 1.436, Validation Accuracy: 47.77%
Epoch [4/10], Training Loss: 1.425, Validation Accuracy: 47.99%
Epoch [5/10], Training Loss: 1.413, Validation Accuracy: 47.48%
Epoch [6/10], Training Loss: 1.402, Validation Accuracy: 48.58%
Epoch [7/10], Training Loss: 1.392, Validation Accuracy: 48.21%
Epoch [8/10], Training Loss: 1.387, Validation Accuracy: 48.54%
Epoch [9/10], Training Loss: 1.380, Validation Accuracy: 48.91%
Epoch [10/10], Training Loss: 1.380, Validation Accuracy: 48.47%
Epoch [1/10], Training Loss: 1.436, Validation Accuracy: 48.06%
Epoch [2/10], Training Loss: 1.420, Validation Accuracy: 48.63%
Epoch [3/10], Training Loss: 1.406, Validation Accuracy: 49.20%
Epoch [4/10], Training Loss: 1.395, Validation Accuracy: 49.50%
Epoch [5/10], Training Loss: 1.391, Validation Accuracy: 48.86%
Epoch [6/10], Training Loss: 1.372, Validation Accuracy: 49.76%
Epoch [7/10], Training Loss: 1.364, Validation Accuracy: 48.91%
Epoch [8/10], Training Loss: 1.356, Validation Accuracy: 49.95%
Epoch [9/10], Training Loss: 1.352, Validation Accuracy: 50.12%
Epoch [10/10], Training Loss: 1.347, Validation Accuracy: 50.05%
Epoch [1/10], Training Loss: 1.380, Validation Accuracy: 50.41%
Epoch [2/10], Training Loss: 1.371, Validation Accuracy: 50.23%
Epoch [3/10], Training Loss: 1.349, Validation Accuracy: 50.32%
Epoch [4/10], Training Loss: 1.338, Validation Accuracy: 51.08%
Epoch [5/10], Training Loss: 1.324, Validation Accuracy: 50.42%
Epoch [6/10], Training Loss: 1.314, Validation Accuracy: 50.99%
Epoch [7/10], Training Loss: 1.307, Validation Accuracy: 51.32%
Epoch [8/10], Training Loss: 1.292, Validation Accuracy: 50.97%
Epoch [9/10], Training Loss: 1.292, Validation Accuracy: 50.33%
Epoch [10/10], Training Loss: 1.284, Validation Accuracy: 51.19%
Epoch [1/10], Training Loss: 1.365, Validation Accuracy: 51.26%
Epoch [2/10], Training Loss: 1.349, Validation Accuracy: 51.78%
Epoch [3/10], Training Loss: 1.329, Validation Accuracy: 51.75%
Epoch [4/10], Training Loss: 1.317, Validation Accuracy: 52.36%
Epoch [5/10], Training Loss: 1.304, Validation Accuracy: 51.42%
Epoch [6/10], Training Loss: 1.299, Validation Accuracy: 51.81%
Epoch [7/10], Training Loss: 1.296, Validation Accuracy: 52.76%
Epoch [8/10], Training Loss: 1.276, Validation Accuracy: 51.78%
Epoch [9/10], Training Loss: 1.271, Validation Accuracy: 52.20%
Epoch [10/10], Training Loss: 1.258, Validation Accuracy: 52.87%
Epoch [1/10], Training Loss: 1.315, Validation Accuracy: 53.29%
Epoch [2/10], Training Loss: 1.291, Validation Accuracy: 52.87%
Epoch [3/10], Training Loss: 1.276, Validation Accuracy: 53.04%
Epoch [4/10], Training Loss: 1.265, Validation Accuracy: 53.18%
Epoch [5/10], Training Loss: 1.247, Validation Accuracy: 53.23%
Epoch [6/10], Training Loss: 1.241, Validation Accuracy: 53.85%
Epoch [7/10], Training Loss: 1.241, Validation Accuracy: 52.45%
Epoch [8/10], Training Loss: 1.221, Validation Accuracy: 53.09%
Epoch [9/10], Training Loss: 1.214, Validation Accuracy: 53.10%
Epoch [10/10], Training Loss: 1.204, Validation Accuracy: 53.54%
Epoch [1/10], Training Loss: 1.287, Validation Accuracy: 53.70%
Epoch [2/10], Training Loss: 1.265, Validation Accuracy: 53.76%
Epoch [3/10], Training Loss: 1.245, Validation Accuracy: 54.55%
Epoch [4/10], Training Loss: 1.232, Validation Accuracy: 53.95%
Epoch [5/10], Training Loss: 1.227, Validation Accuracy: 54.54%
Epoch [6/10], Training Loss: 1.212, Validation Accuracy: 53.97%
Epoch [7/10], Training Loss: 1.205, Validation Accuracy: 54.12%
Epoch [8/10], Training Loss: 1.198, Validation Accuracy: 54.48%
Epoch [9/10], Training Loss: 1.176, Validation Accuracy: 54.68%
Epoch [10/10], Training Loss: 1.178, Validation Accuracy: 54.60%
Epoch [1/10], Training Loss: 1.273, Validation Accuracy: 55.29%
Epoch [2/10], Training Loss: 1.252, Validation Accuracy: 54.75%
Epoch [3/10], Training Loss: 1.223, Validation Accuracy: 55.00%
Epoch [4/10], Training Loss: 1.208, Validation Accuracy: 55.70%
Epoch [5/10], Training Loss: 1.204, Validation Accuracy: 54.37%
Epoch [6/10], Training Loss: 1.188, Validation Accuracy: 55.07%
Epoch [7/10], Training Loss: 1.175, Validation Accuracy: 54.31%
Epoch [8/10], Training Loss: 1.167, Validation Accuracy: 53.84%
Epoch [9/10], Training Loss: 1.165, Validation Accuracy: 54.76%
Epoch [10/10], Training Loss: 1.152, Validation Accuracy: 55.53%
Epoch [1/10], Training Loss: 1.224, Validation Accuracy: 54.93%
Epoch [2/10], Training Loss: 1.214, Validation Accuracy: 55.09%
Epoch [3/10], Training Loss: 1.187, Validation Accuracy: 55.64%
Epoch [4/10], Training Loss: 1.165, Validation Accuracy: 55.28%
Epoch [5/10], Training Loss: 1.161, Validation Accuracy: 55.76%
Epoch [6/10], Training Loss: 1.148, Validation Accuracy: 55.59%
Epoch [7/10], Training Loss: 1.138, Validation Accuracy: 55.62%
Epoch [8/10], Training Loss: 1.120, Validation Accuracy: 55.04%
Epoch [9/10], Training Loss: 1.118, Validation Accuracy: 54.95%
Epoch [10/10], Training Loss: 1.107, Validation Accuracy: 55.78%
Epoch [1/10], Training Loss: 1.217, Validation Accuracy: 56.27%
Epoch [2/10], Training Loss: 1.188, Validation Accuracy: 56.36%
Epoch [3/10], Training Loss: 1.174, Validation Accuracy: 56.14%
Epoch [4/10], Training Loss: 1.152, Validation Accuracy: 55.94%
Epoch [5/10], Training Loss: 1.145, Validation Accuracy: 56.01%
Epoch [6/10], Training Loss: 1.122, Validation Accuracy: 57.43%
Epoch [7/10], Training Loss: 1.108, Validation Accuracy: 56.87%
Epoch [8/10], Training Loss: 1.103, Validation Accuracy: 56.61%
Epoch [9/10], Training Loss: 1.096, Validation Accuracy: 57.07%
Epoch [10/10], Training Loss: 1.078, Validation Accuracy: 56.76%
Epoch [1/10], Training Loss: 1.189, Validation Accuracy: 56.21%
Epoch [2/10], Training Loss: 1.154, Validation Accuracy: 56.95%
Epoch [3/10], Training Loss: 1.134, Validation Accuracy: 57.53%
Epoch [4/10], Training Loss: 1.109, Validation Accuracy: 57.13%
Epoch [5/10], Training Loss: 1.098, Validation Accuracy: 56.51%
Epoch [6/10], Training Loss: 1.088, Validation Accuracy: 56.51%
Epoch [7/10], Training Loss: 1.075, Validation Accuracy: 56.96%
Epoch [8/10], Training Loss: 1.063, Validation Accuracy: 57.04%
Epoch [9/10], Training Loss: 1.065, Validation Accuracy: 57.59%
Epoch [10/10], Training Loss: 1.041, Validation Accuracy: 56.91%
Epoch [1/10], Training Loss: 1.169, Validation Accuracy: 58.49%
Epoch [2/10], Training Loss: 1.134, Validation Accuracy: 58.64%
Epoch [3/10], Training Loss: 1.123, Validation Accuracy: 58.37%
Epoch [4/10], Training Loss: 1.103, Validation Accuracy: 57.60%
Epoch [5/10], Training Loss: 1.091, Validation Accuracy: 58.06%
Epoch [6/10], Training Loss: 1.069, Validation Accuracy: 57.85%
Epoch [7/10], Training Loss: 1.055, Validation Accuracy: 57.64%
Epoch [8/10], Training Loss: 1.056, Validation Accuracy: 58.14%
Epoch [9/10], Training Loss: 1.041, Validation Accuracy: 57.86%
Epoch [10/10], Training Loss: 1.019, Validation Accuracy: 57.76%
Epoch [1/10], Training Loss: 1.164, Validation Accuracy: 58.04%
Epoch [2/10], Training Loss: 1.136, Validation Accuracy: 57.47%
Epoch [3/10], Training Loss: 1.106, Validation Accuracy: 58.35%
Epoch [4/10], Training Loss: 1.088, Validation Accuracy: 58.07%
Epoch [5/10], Training Loss: 1.077, Validation Accuracy: 57.62%
Epoch [6/10], Training Loss: 1.064, Validation Accuracy: 57.01%
Epoch [7/10], Training Loss: 1.056, Validation Accuracy: 58.23%
Epoch [8/10], Training Loss: 1.031, Validation Accuracy: 58.10%
Epoch [9/10], Training Loss: 1.027, Validation Accuracy: 57.88%
Epoch [10/10], Training Loss: 1.016, Validation Accuracy: 57.91%
Epoch [1/10], Training Loss: 1.136, Validation Accuracy: 58.20%
Epoch [2/10], Training Loss: 1.090, Validation Accuracy: 58.08%
Epoch [3/10], Training Loss: 1.076, Validation Accuracy: 58.71%
Epoch [4/10], Training Loss: 1.052, Validation Accuracy: 58.28%
Epoch [5/10], Training Loss: 1.037, Validation Accuracy: 58.42%
Epoch [6/10], Training Loss: 1.021, Validation Accuracy: 57.75%
Epoch [7/10], Training Loss: 1.005, Validation Accuracy: 58.23%
Epoch [8/10], Training Loss: 1.001, Validation Accuracy: 58.11%
Epoch [9/10], Training Loss: 0.980, Validation Accuracy: 58.60%
Epoch [10/10], Training Loss: 0.971, Validation Accuracy: 58.27%
Epoch [1/10], Training Loss: 1.120, Validation Accuracy: 58.83%
Epoch [2/10], Training Loss: 1.075, Validation Accuracy: 58.78%
Epoch [3/10], Training Loss: 1.045, Validation Accuracy: 58.83%
Epoch [4/10], Training Loss: 1.034, Validation Accuracy: 58.77%
Epoch [5/10], Training Loss: 1.022, Validation Accuracy: 58.79%
Epoch [6/10], Training Loss: 0.998, Validation Accuracy: 59.62%
Epoch [7/10], Training Loss: 0.981, Validation Accuracy: 59.44%
Epoch [8/10], Training Loss: 0.970, Validation Accuracy: 59.57%
Epoch [9/10], Training Loss: 0.958, Validation Accuracy: 59.61%
Epoch [10/10], Training Loss: 0.939, Validation Accuracy: 58.96%
Epoch [1/10], Training Loss: 1.094, Validation Accuracy: 58.33%
Epoch [2/10], Training Loss: 1.046, Validation Accuracy: 59.36%
Epoch [3/10], Training Loss: 1.016, Validation Accuracy: 59.59%
Epoch [4/10], Training Loss: 0.994, Validation Accuracy: 59.25%
Epoch [5/10], Training Loss: 0.978, Validation Accuracy: 59.34%
Epoch [6/10], Training Loss: 0.958, Validation Accuracy: 59.11%
Epoch [7/10], Training Loss: 0.948, Validation Accuracy: 59.38%
Epoch [8/10], Training Loss: 0.942, Validation Accuracy: 59.17%
Epoch [9/10], Training Loss: 0.923, Validation Accuracy: 59.02%
Epoch [10/10], Training Loss: 0.918, Validation Accuracy: 59.14%
Epoch [1/10], Training Loss: 1.087, Validation Accuracy: 58.87%
Epoch [2/10], Training Loss: 1.048, Validation Accuracy: 59.53%
Epoch [3/10], Training Loss: 1.013, Validation Accuracy: 59.54%
Epoch [4/10], Training Loss: 0.992, Validation Accuracy: 60.34%
Epoch [5/10], Training Loss: 0.972, Validation Accuracy: 59.75%
Epoch [6/10], Training Loss: 0.958, Validation Accuracy: 59.86%
Epoch [7/10], Training Loss: 0.948, Validation Accuracy: 59.35%
Epoch [8/10], Training Loss: 0.927, Validation Accuracy: 59.92%
Epoch [9/10], Training Loss: 0.912, Validation Accuracy: 59.09%
Epoch [10/10], Training Loss: 0.898, Validation Accuracy: 59.07%
Epoch [1/10], Training Loss: 1.092, Validation Accuracy: 59.31%
Epoch [2/10], Training Loss: 1.038, Validation Accuracy: 59.15%
Epoch [3/10], Training Loss: 1.011, Validation Accuracy: 60.16%
Epoch [4/10], Training Loss: 0.989, Validation Accuracy: 59.93%
Epoch [5/10], Training Loss: 0.965, Validation Accuracy: 59.13%
Epoch [6/10], Training Loss: 0.959, Validation Accuracy: 59.23%
Epoch [7/10], Training Loss: 0.943, Validation Accuracy: 59.43%
Epoch [8/10], Training Loss: 0.920, Validation Accuracy: 58.55%
Epoch [9/10], Training Loss: 0.913, Validation Accuracy: 59.34%
Epoch [10/10], Training Loss: 0.892, Validation Accuracy: 59.54%
Epoch [1/10], Training Loss: 1.061, Validation Accuracy: 60.01%
Epoch [2/10], Training Loss: 1.007, Validation Accuracy: 59.32%
Epoch [3/10], Training Loss: 0.983, Validation Accuracy: 59.24%
Epoch [4/10], Training Loss: 0.954, Validation Accuracy: 59.25%
Epoch [5/10], Training Loss: 0.933, Validation Accuracy: 60.02%
Epoch [6/10], Training Loss: 0.915, Validation Accuracy: 59.07%
Epoch [7/10], Training Loss: 0.901, Validation Accuracy: 60.05%
Epoch [8/10], Training Loss: 0.883, Validation Accuracy: 59.95%
Epoch [9/10], Training Loss: 0.869, Validation Accuracy: 59.56%
Epoch [10/10], Training Loss: 0.863, Validation Accuracy: 58.65%
Epoch [1/10], Training Loss: 1.040, Validation Accuracy: 59.49%
Epoch [2/10], Training Loss: 0.971, Validation Accuracy: 59.63%
Epoch [3/10], Training Loss: 0.956, Validation Accuracy: 59.94%
Epoch [4/10], Training Loss: 0.930, Validation Accuracy: 60.40%
Epoch [5/10], Training Loss: 0.910, Validation Accuracy: 60.58%
Epoch [6/10], Training Loss: 0.898, Validation Accuracy: 60.69%
Epoch [7/10], Training Loss: 0.866, Validation Accuracy: 59.35%
Epoch [8/10], Training Loss: 0.853, Validation Accuracy: 60.01%
Epoch [9/10], Training Loss: 0.841, Validation Accuracy: 60.45%
Epoch [10/10], Training Loss: 0.821, Validation Accuracy: 60.03%
Epoch [1/10], Training Loss: 1.014, Validation Accuracy: 60.72%
Epoch [2/10], Training Loss: 0.961, Validation Accuracy: 60.33%
Epoch [3/10], Training Loss: 0.933, Validation Accuracy: 60.19%
Epoch [4/10], Training Loss: 0.894, Validation Accuracy: 60.75%
Epoch [5/10], Training Loss: 0.876, Validation Accuracy: 60.68%
Epoch [6/10], Training Loss: 0.857, Validation Accuracy: 60.13%
Epoch [7/10], Training Loss: 0.832, Validation Accuracy: 60.72%
Epoch [8/10], Training Loss: 0.831, Validation Accuracy: 60.43%
Epoch [9/10], Training Loss: 0.804, Validation Accuracy: 59.81%
Epoch [10/10], Training Loss: 0.802, Validation Accuracy: 59.95%
Epoch [1/10], Training Loss: 1.020, Validation Accuracy: 60.85%
Epoch [2/10], Training Loss: 0.963, Validation Accuracy: 60.84%
Epoch [3/10], Training Loss: 0.935, Validation Accuracy: 60.40%
Epoch [4/10], Training Loss: 0.901, Validation Accuracy: 61.05%
Epoch [5/10], Training Loss: 0.871, Validation Accuracy: 60.64%
Epoch [6/10], Training Loss: 0.852, Validation Accuracy: 60.71%
Epoch [7/10], Training Loss: 0.838, Validation Accuracy: 59.59%
Epoch [8/10], Training Loss: 0.825, Validation Accuracy: 59.95%
Epoch [9/10], Training Loss: 0.813, Validation Accuracy: 60.26%
Epoch [10/10], Training Loss: 0.791, Validation Accuracy: 60.29%
Epoch [1/10], Training Loss: 1.027, Validation Accuracy: 60.35%
Epoch [2/10], Training Loss: 0.963, Validation Accuracy: 59.95%
Epoch [3/10], Training Loss: 0.936, Validation Accuracy: 60.04%
Epoch [4/10], Training Loss: 0.905, Validation Accuracy: 59.27%
Epoch [5/10], Training Loss: 0.878, Validation Accuracy: 60.65%
Epoch [6/10], Training Loss: 0.854, Validation Accuracy: 60.69%
Epoch [7/10], Training Loss: 0.835, Validation Accuracy: 60.70%
Epoch [8/10], Training Loss: 0.815, Validation Accuracy: 60.20%
Epoch [9/10], Training Loss: 0.806, Validation Accuracy: 59.93%
Epoch [10/10], Training Loss: 0.789, Validation Accuracy: 60.01%
Epoch [1/10], Training Loss: 1.006, Validation Accuracy: 60.20%
Epoch [2/10], Training Loss: 0.939, Validation Accuracy: 61.24%
Epoch [3/10], Training Loss: 0.900, Validation Accuracy: 60.77%
Epoch [4/10], Training Loss: 0.865, Validation Accuracy: 61.03%
Epoch [5/10], Training Loss: 0.833, Validation Accuracy: 60.95%
Epoch [6/10], Training Loss: 0.814, Validation Accuracy: 61.22%
Epoch [7/10], Training Loss: 0.797, Validation Accuracy: 60.94%
Epoch [8/10], Training Loss: 0.782, Validation Accuracy: 60.43%
Epoch [9/10], Training Loss: 0.762, Validation Accuracy: 60.74%
Epoch [10/10], Training Loss: 0.746, Validation Accuracy: 59.56%
Epoch [1/10], Training Loss: 0.968, Validation Accuracy: 60.43%
Epoch [2/10], Training Loss: 0.899, Validation Accuracy: 60.32%
Epoch [3/10], Training Loss: 0.866, Validation Accuracy: 60.14%
Epoch [4/10], Training Loss: 0.826, Validation Accuracy: 61.42%
Epoch [5/10], Training Loss: 0.804, Validation Accuracy: 60.90%
Epoch [6/10], Training Loss: 0.782, Validation Accuracy: 60.88%
Epoch [7/10], Training Loss: 0.769, Validation Accuracy: 61.04%
Epoch [8/10], Training Loss: 0.755, Validation Accuracy: 61.31%
Epoch [9/10], Training Loss: 0.729, Validation Accuracy: 60.47%
Epoch [10/10], Training Loss: 0.710, Validation Accuracy: 61.22%
Epoch [1/10], Training Loss: 0.963, Validation Accuracy: 60.92%
Epoch [2/10], Training Loss: 0.884, Validation Accuracy: 61.44%
Epoch [3/10], Training Loss: 0.842, Validation Accuracy: 60.41%
Epoch [4/10], Training Loss: 0.806, Validation Accuracy: 61.08%
Epoch [5/10], Training Loss: 0.784, Validation Accuracy: 60.89%
Epoch [6/10], Training Loss: 0.767, Validation Accuracy: 60.89%
Epoch [7/10], Training Loss: 0.756, Validation Accuracy: 60.06%
Epoch [8/10], Training Loss: 0.731, Validation Accuracy: 60.81%
Epoch [9/10], Training Loss: 0.704, Validation Accuracy: 60.49%
Epoch [10/10], Training Loss: 0.695, Validation Accuracy: 60.05%
Epoch [1/10], Training Loss: 0.964, Validation Accuracy: 60.51%
Epoch [2/10], Training Loss: 0.883, Validation Accuracy: 61.49%
Epoch [3/10], Training Loss: 0.843, Validation Accuracy: 60.38%
Epoch [4/10], Training Loss: 0.816, Validation Accuracy: 61.64%
Epoch [5/10], Training Loss: 0.777, Validation Accuracy: 61.64%
Epoch [6/10], Training Loss: 0.759, Validation Accuracy: 61.07%
Epoch [7/10], Training Loss: 0.733, Validation Accuracy: 60.82%
Epoch [8/10], Training Loss: 0.725, Validation Accuracy: 60.75%
Epoch [9/10], Training Loss: 0.693, Validation Accuracy: 60.64%
Epoch [10/10], Training Loss: 0.683, Validation Accuracy: 60.95%
Epoch [1/10], Training Loss: 0.968, Validation Accuracy: 61.12%
Epoch [2/10], Training Loss: 0.886, Validation Accuracy: 61.27%
Epoch [3/10], Training Loss: 0.838, Validation Accuracy: 60.61%
Epoch [4/10], Training Loss: 0.811, Validation Accuracy: 60.42%
Epoch [5/10], Training Loss: 0.776, Validation Accuracy: 60.82%
Epoch [6/10], Training Loss: 0.753, Validation Accuracy: 59.99%
Epoch [7/10], Training Loss: 0.740, Validation Accuracy: 60.57%
Epoch [8/10], Training Loss: 0.716, Validation Accuracy: 60.30%
Epoch [9/10], Training Loss: 0.701, Validation Accuracy: 61.07%
Epoch [10/10], Training Loss: 0.677, Validation Accuracy: 60.54%
Epoch [1/10], Training Loss: 0.956, Validation Accuracy: 60.96%
Epoch [2/10], Training Loss: 0.862, Validation Accuracy: 61.37%
Epoch [3/10], Training Loss: 0.819, Validation Accuracy: 60.80%
Epoch [4/10], Training Loss: 0.787, Validation Accuracy: 61.54%
Epoch [5/10], Training Loss: 0.759, Validation Accuracy: 61.26%
Epoch [6/10], Training Loss: 0.729, Validation Accuracy: 61.54%
Epoch [7/10], Training Loss: 0.708, Validation Accuracy: 61.38%
Epoch [8/10], Training Loss: 0.682, Validation Accuracy: 61.00%
Epoch [9/10], Training Loss: 0.666, Validation Accuracy: 61.08%
Epoch [10/10], Training Loss: 0.643, Validation Accuracy: 60.62%
Epoch [1/10], Training Loss: 0.917, Validation Accuracy: 60.73%
Epoch [2/10], Training Loss: 0.834, Validation Accuracy: 60.89%
Epoch [3/10], Training Loss: 0.786, Validation Accuracy: 61.57%
Epoch [4/10], Training Loss: 0.739, Validation Accuracy: 60.83%
Epoch [5/10], Training Loss: 0.723, Validation Accuracy: 61.18%
Epoch [6/10], Training Loss: 0.696, Validation Accuracy: 60.77%
Epoch [7/10], Training Loss: 0.670, Validation Accuracy: 61.02%
Epoch [8/10], Training Loss: 0.655, Validation Accuracy: 61.36%
Epoch [9/10], Training Loss: 0.627, Validation Accuracy: 61.39%
Epoch [10/10], Training Loss: 0.604, Validation Accuracy: 59.89%
Epoch [1/10], Training Loss: 0.914, Validation Accuracy: 60.25%
Epoch [2/10], Training Loss: 0.823, Validation Accuracy: 60.56%
Epoch [3/10], Training Loss: 0.768, Validation Accuracy: 61.43%
Epoch [4/10], Training Loss: 0.726, Validation Accuracy: 60.62%
Epoch [5/10], Training Loss: 0.696, Validation Accuracy: 61.22%
Epoch [6/10], Training Loss: 0.665, Validation Accuracy: 60.97%
Epoch [7/10], Training Loss: 0.650, Validation Accuracy: 61.21%
Epoch [8/10], Training Loss: 0.624, Validation Accuracy: 60.95%
Epoch [9/10], Training Loss: 0.602, Validation Accuracy: 60.34%
Epoch [10/10], Training Loss: 0.594, Validation Accuracy: 60.41%
Epoch [1/10], Training Loss: 0.909, Validation Accuracy: 60.80%
Epoch [2/10], Training Loss: 0.811, Validation Accuracy: 61.29%
Epoch [3/10], Training Loss: 0.759, Validation Accuracy: 60.89%
Epoch [4/10], Training Loss: 0.724, Validation Accuracy: 61.71%
Epoch [5/10], Training Loss: 0.689, Validation Accuracy: 60.79%
Epoch [6/10], Training Loss: 0.661, Validation Accuracy: 61.62%
Epoch [7/10], Training Loss: 0.634, Validation Accuracy: 60.79%
Epoch [8/10], Training Loss: 0.610, Validation Accuracy: 61.40%
Epoch [9/10], Training Loss: 0.593, Validation Accuracy: 61.25%
Epoch [10/10], Training Loss: 0.566, Validation Accuracy: 61.04%
Epoch [1/10], Training Loss: 0.917, Validation Accuracy: 60.59%
Epoch [2/10], Training Loss: 0.821, Validation Accuracy: 60.63%
Epoch [3/10], Training Loss: 0.758, Validation Accuracy: 60.42%
Epoch [4/10], Training Loss: 0.719, Validation Accuracy: 61.55%
Epoch [5/10], Training Loss: 0.685, Validation Accuracy: 60.88%
Epoch [6/10], Training Loss: 0.660, Validation Accuracy: 60.46%
Epoch [7/10], Training Loss: 0.632, Validation Accuracy: 60.73%
Epoch [8/10], Training Loss: 0.608, Validation Accuracy: 60.95%
Epoch [9/10], Training Loss: 0.583, Validation Accuracy: 60.34%
Epoch [10/10], Training Loss: 0.571, Validation Accuracy: 60.70%
Epoch [1/10], Training Loss: 0.906, Validation Accuracy: 61.02%
Epoch [2/10], Training Loss: 0.802, Validation Accuracy: 61.09%
Epoch [3/10], Training Loss: 0.744, Validation Accuracy: 61.23%
Epoch [4/10], Training Loss: 0.696, Validation Accuracy: 61.02%
Epoch [5/10], Training Loss: 0.660, Validation Accuracy: 61.50%
Epoch [6/10], Training Loss: 0.630, Validation Accuracy: 60.77%
Epoch [7/10], Training Loss: 0.614, Validation Accuracy: 60.84%
Epoch [8/10], Training Loss: 0.590, Validation Accuracy: 60.28%
Epoch [9/10], Training Loss: 0.566, Validation Accuracy: 60.62%
Epoch [10/10], Training Loss: 0.543, Validation Accuracy: 60.68%
Epoch [1/10], Training Loss: 0.884, Validation Accuracy: 59.64%
Epoch [2/10], Training Loss: 0.772, Validation Accuracy: 61.15%
Epoch [3/10], Training Loss: 0.701, Validation Accuracy: 61.02%
Epoch [4/10], Training Loss: 0.674, Validation Accuracy: 60.84%
Epoch [5/10], Training Loss: 0.624, Validation Accuracy: 61.50%
Epoch [6/10], Training Loss: 0.596, Validation Accuracy: 61.22%
Epoch [7/10], Training Loss: 0.571, Validation Accuracy: 61.13%
Epoch [8/10], Training Loss: 0.552, Validation Accuracy: 61.01%
Epoch [9/10], Training Loss: 0.540, Validation Accuracy: 60.99%
Epoch [10/10], Training Loss: 0.518, Validation Accuracy: 60.74%
Epoch [1/10], Training Loss: 0.872, Validation Accuracy: 60.96%
Epoch [2/10], Training Loss: 0.751, Validation Accuracy: 61.20%
Epoch [3/10], Training Loss: 0.698, Validation Accuracy: 60.83%
Epoch [4/10], Training Loss: 0.648, Validation Accuracy: 61.19%
Epoch [5/10], Training Loss: 0.614, Validation Accuracy: 61.31%
Epoch [6/10], Training Loss: 0.571, Validation Accuracy: 61.34%
Epoch [7/10], Training Loss: 0.552, Validation Accuracy: 60.91%
Epoch [8/10], Training Loss: 0.532, Validation Accuracy: 61.43%
Epoch [9/10], Training Loss: 0.509, Validation Accuracy: 60.69%
Epoch [10/10], Training Loss: 0.490, Validation Accuracy: 60.44%
Epoch [1/10], Training Loss: 0.869, Validation Accuracy: 60.76%
Epoch [2/10], Training Loss: 0.745, Validation Accuracy: 60.33%
Epoch [3/10], Training Loss: 0.687, Validation Accuracy: 61.01%
Epoch [4/10], Training Loss: 0.636, Validation Accuracy: 60.98%
Epoch [5/10], Training Loss: 0.600, Validation Accuracy: 61.47%
Epoch [6/10], Training Loss: 0.562, Validation Accuracy: 60.92%
Epoch [7/10], Training Loss: 0.545, Validation Accuracy: 61.18%
Epoch [8/10], Training Loss: 0.512, Validation Accuracy: 61.06%
Epoch [9/10], Training Loss: 0.492, Validation Accuracy: 61.12%
Epoch [10/10], Training Loss: 0.476, Validation Accuracy: 60.86%
Epoch [1/10], Training Loss: 0.887, Validation Accuracy: 60.19%
Epoch [2/10], Training Loss: 0.758, Validation Accuracy: 60.35%
Epoch [3/10], Training Loss: 0.673, Validation Accuracy: 60.70%
Epoch [4/10], Training Loss: 0.629, Validation Accuracy: 60.82%
Epoch [5/10], Training Loss: 0.592, Validation Accuracy: 60.22%
Epoch [6/10], Training Loss: 0.567, Validation Accuracy: 59.95%
Epoch [7/10], Training Loss: 0.539, Validation Accuracy: 59.76%
Epoch [8/10], Training Loss: 0.525, Validation Accuracy: 59.91%
Epoch [9/10], Training Loss: 0.491, Validation Accuracy: 60.32%
Epoch [10/10], Training Loss: 0.470, Validation Accuracy: 59.88%
Epoch [1/10], Training Loss: 0.873, Validation Accuracy: 60.29%
Epoch [2/10], Training Loss: 0.731, Validation Accuracy: 59.86%
Epoch [3/10], Training Loss: 0.659, Validation Accuracy: 60.12%
Epoch [4/10], Training Loss: 0.616, Validation Accuracy: 61.04%
Epoch [5/10], Training Loss: 0.577, Validation Accuracy: 60.94%
Epoch [6/10], Training Loss: 0.537, Validation Accuracy: 61.07%
Epoch [7/10], Training Loss: 0.511, Validation Accuracy: 60.45%
Epoch [8/10], Training Loss: 0.491, Validation Accuracy: 60.12%
Epoch [9/10], Training Loss: 0.473, Validation Accuracy: 60.40%
Epoch [10/10], Training Loss: 0.447, Validation Accuracy: 60.30%
Epoch [1/10], Training Loss: 0.846, Validation Accuracy: 60.81%
Epoch [2/10], Training Loss: 0.719, Validation Accuracy: 60.84%
Epoch [3/10], Training Loss: 0.644, Validation Accuracy: 60.63%
Epoch [4/10], Training Loss: 0.587, Validation Accuracy: 60.83%
Epoch [5/10], Training Loss: 0.553, Validation Accuracy: 61.01%
Epoch [6/10], Training Loss: 0.517, Validation Accuracy: 61.11%
Epoch [7/10], Training Loss: 0.487, Validation Accuracy: 59.88%
Epoch [8/10], Training Loss: 0.461, Validation Accuracy: 61.19%
Epoch [9/10], Training Loss: 0.441, Validation Accuracy: 60.51%
Epoch [10/10], Training Loss: 0.424, Validation Accuracy: 60.53%
Epoch [1/10], Training Loss: 0.825, Validation Accuracy: 59.82%
Epoch [2/10], Training Loss: 0.702, Validation Accuracy: 60.44%
Epoch [3/10], Training Loss: 0.622, Validation Accuracy: 61.11%
Epoch [4/10], Training Loss: 0.566, Validation Accuracy: 60.49%
Epoch [5/10], Training Loss: 0.523, Validation Accuracy: 60.87%
Epoch [6/10], Training Loss: 0.498, Validation Accuracy: 61.01%
Epoch [7/10], Training Loss: 0.462, Validation Accuracy: 60.51%
Epoch [8/10], Training Loss: 0.444, Validation Accuracy: 60.70%
Epoch [9/10], Training Loss: 0.422, Validation Accuracy: 59.57%
Epoch [10/10], Training Loss: 0.399, Validation Accuracy: 60.21%
Epoch [1/10], Training Loss: 0.841, Validation Accuracy: 60.28%
Epoch [2/10], Training Loss: 0.680, Validation Accuracy: 60.49%
Epoch [3/10], Training Loss: 0.617, Validation Accuracy: 60.83%
Epoch [4/10], Training Loss: 0.548, Validation Accuracy: 60.64%
Epoch [5/10], Training Loss: 0.512, Validation Accuracy: 60.62%
Epoch [6/10], Training Loss: 0.487, Validation Accuracy: 60.64%
Epoch [7/10], Training Loss: 0.453, Validation Accuracy: 61.14%
Epoch [8/10], Training Loss: 0.424, Validation Accuracy: 60.02%
Epoch [9/10], Training Loss: 0.398, Validation Accuracy: 61.16%
Epoch [10/10], Training Loss: 0.386, Validation Accuracy: 60.43%
Epoch [1/10], Training Loss: 0.853, Validation Accuracy: 59.42%
Epoch [2/10], Training Loss: 0.674, Validation Accuracy: 60.14%
Epoch [3/10], Training Loss: 0.612, Validation Accuracy: 60.24%
Epoch [4/10], Training Loss: 0.544, Validation Accuracy: 59.83%
Epoch [5/10], Training Loss: 0.506, Validation Accuracy: 60.75%
Epoch [6/10], Training Loss: 0.479, Validation Accuracy: 60.10%
Epoch [7/10], Training Loss: 0.446, Validation Accuracy: 60.51%
Epoch [8/10], Training Loss: 0.417, Validation Accuracy: 60.20%
Epoch [9/10], Training Loss: 0.397, Validation Accuracy: 60.17%
Epoch [10/10], Training Loss: 0.374, Validation Accuracy: 60.54%
"""

# Regular expression to find validation accuracies
accuracies = re.findall(r'Validation Accuracy: (\d+\.\d+)%', log)

# Convert accuracies from string to float
accuracies = [float(acc) for acc in accuracies]

# Print accuracies
print("Accuracies:", accuracies)

# Print size of the array
print("Size of array:", len(accuracies))


Accuracies: [10.38, 10.94, 11.72, 11.68, 11.6, 11.38, 11.52, 12.27, 13.94, 15.19, 15.12, 15.61, 16.64, 17.53, 19.21, 21.53, 23.01, 23.34, 24.25, 25.36, 26.3, 26.39, 27.61, 28.12, 28.83, 29.58, 30.28, 30.92, 31.31, 32.0, 32.39, 33.49, 34.76, 35.22, 35.6, 33.84, 35.93, 36.39, 37.76, 37.77, 38.29, 38.36, 38.67, 39.58, 39.57, 40.46, 40.25, 40.88, 41.43, 40.36, 41.45, 41.22, 41.71, 42.18, 41.95, 43.21, 42.77, 42.95, 42.75, 43.66, 43.44, 44.21, 44.34, 44.06, 44.8, 45.22, 45.04, 45.5, 44.99, 45.8, 46.15, 46.18, 45.19, 46.26, 46.43, 47.02, 47.07, 47.26, 46.92, 47.22, 46.9, 47.72, 47.77, 47.99, 47.48, 48.58, 48.21, 48.54, 48.91, 48.47, 48.06, 48.63, 49.2, 49.5, 48.86, 49.76, 48.91, 49.95, 50.12, 50.05, 50.41, 50.23, 50.32, 51.08, 50.42, 50.99, 51.32, 50.97, 50.33, 51.19, 51.26, 51.78, 51.75, 52.36, 51.42, 51.81, 52.76, 51.78, 52.2, 52.87, 53.29, 52.87, 53.04, 53.18, 53.23, 53.85, 52.45, 53.09, 53.1, 53.54, 53.7, 53.76, 54.55, 53.95, 54.54, 53.97, 54.12, 54.48, 54.68, 54.6, 55.29, 54.75, 55.0, 5

In [ ]:
import numpy as np
from scipy.stats import truncnorm
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch.utils.data import Subset, DataLoader
from torchvision.datasets import CIFAR10
from typing import Dict, Tuple
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import random

# Define VAE model
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim, z_dim):
        super(VAE, self).__init__()
        self.x_dim = x_dim
        self.h_dim = h_dim
        self.z_dim = z_dim

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, stride=2, padding=1),  # 16x16
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2, padding=1),  # 8x8
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),  # 4x4
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(2048, 256),
            nn.ReLU(),
            nn.Linear(256, 2*z_dim)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 2048),
            nn.ReLU(),
            nn.Unflatten(1, (128, 4, 4)),
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),  # 8x8
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1),  # 16x16
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 4, stride=2, padding=1),  # 32x32
            nn.Sigmoid()
        )

    def encode(self, x):
        h = self.encoder(x)
        mu, logvar = h[:, :self.z_dim], h[:, self.z_dim:]
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


# Define VAE training procedure
def vae_train(vae: VAE, trainloader: DataLoader, epochs: int) -> None:
    optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

    for epoch in range(epochs):
        vae.train()
        for i, data in enumerate(trainloader, 0):
            inputs, _ = data
            optimizer.zero_grad()
            recon_x, mu, logvar = vae(inputs)
            loss = vae_loss(recon_x, inputs, mu, logvar)
            loss.backward()
            optimizer.step()

        scheduler.step()


# Define classification model
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Load CIFAR10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

full_dataset = CIFAR10(root="./data", train=True, download=True, transform=transform)
test_set = CIFAR10(root="./data", train=False, download=True, transform=transform)

# Separate dataset by class
class_indices = {i: [] for i in range(10)}  # CIFAR-10 has 10 classes
for idx, (_, label) in enumerate(full_dataset):
    class_indices[label].append(idx)

# Define target count per class, summing to 60,000 with random distribution
class_counts = np.random.multinomial(60000, [0.1] * 10)  # Adjust probabilities if you want specific class biases
print("Random Images per Class:", class_counts)

# Sample indices based on the specified class counts
indices = []
for class_id, count in enumerate(class_counts):
    # Ensure count does not exceed available images
    count = min(count, len(class_indices[class_id]))
    selected_indices = random.sample(class_indices[class_id], count)
    indices.extend(selected_indices)

# Create a custom CIFAR-10 dataset with the sampled indices
custom_dataset = Subset(full_dataset, indices)
# Split dataset into training and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_set, val_set = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# Create training and validation loaders
trainloader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2)
valloader = DataLoader(val_set, batch_size=128, shuffle=False, num_workers=2)
testloader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=2)



# Define VAE loss function
def vae_loss(recon_x, x, mu, logvar):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

# Define training procedure for classification model
def train(net: nn.Module, trainloader: DataLoader, valloader: DataLoader, epochs: int) -> None:
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

    best_acc = 0.0

    for epoch in range(epochs):
        # Training loop
        net.train()
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        scheduler.step()

        # Validation loop
        net.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in valloader:
                images, labels = data
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = 100 * correct / total
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(net.state_dict(), 'best_model.pth')

        print(f"Epoch [{epoch+1}/{epochs}], Training Loss: {running_loss / (i+1):.3f}, Validation Accuracy: {val_acc:.2f}%")

# Define evaluation procedure
def evaluate(net: nn.Module, testloader: DataLoader) -> Tuple[float, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    net.eval()
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    accuracy = 100 * correct / total

    # Calculate confusion matrix
    cm = confusion_matrix(all_labels, all_predictions)
    print(f"Confusion Matrix:\n{cm}")

    # Calculate TP, FP, TN, FN for each class
    tp = np.diag(cm)
    fp = cm.sum(axis=0) - tp
    fn = cm.sum(axis=1) - tp
    tn = cm.sum() - (fp + fn + tp)

    # Calculate precision, recall, and F1 score for each class
    precision = precision_score(all_labels, all_predictions, average=None)
    recall = recall_score(all_labels, all_predictions, average=None)
    f1 = f1_score(all_labels, all_predictions, average=None)

    return accuracy, tp, fp, tn, fn, precision, recall, f1

# Initialize clients
def initialize_clients(trainset, transform, batch_size, num_clients):
    clients = {}
    for i in range(num_clients):
        client_trainset = torch.utils.data.Subset(trainset, range(i * len(trainset) // num_clients, (i + 1) * len(trainset) // num_clients))
        client_trainloader = torch.utils.data.DataLoader(client_trainset, batch_size=batch_size, shuffle=True)
        clients[f"client_{i}"] = client_trainloader
    return clients

def get_distribution_info(vae: VAE) -> Dict:
    # Implement the logic to extract distribution information from the VAE
    # This can involve computing statistics, parameters, or any other relevant information
    # that can be used to generate augmented data

    # Example implementation:
    distribution_info = {

        "normal": {
            "mean": vae.encoder[-1].bias.data.cpu().numpy(),
            "std": torch.exp(0.5 * vae.encoder[-1].weight.data).cpu().numpy()
        }
    }

    return distribution_info

def send_distribution_info(distribution_info: Dict) -> None:
    # Implement the logic to send the distribution information to the global server
    # This can involve using a network protocol, a message queue, or any other communication mechanism
    # to send the information

    # Example implementation:
    # Send the distribution information to the global server using a network protocol
    # For example, you can use the `socket` module to send the information over a network
    # or use a message queue like `RabbitMQ` to send the information
    pass

def generate_augmented_data(vae: VAE, distribution_info_normal: Dict) -> torch.Tensor:
    # Generate augmented data using both uniform and truncated uniform distributions



    mean_normal = distribution_info_normal["mean"]
    std_normal = distribution_info_normal["std"]

    augmented_data_normal = torch.randn(64, vae.z_dim) * std_normal + mean_normal





    # Calculate the average of augmented data from both distributions
    augmented_data_average =  augmented_data_normal

    return augmented_data_average

def federated_train(net: nn.Module, vae: VAE, trainloaders: Dict[str, DataLoader], trainloader: DataLoader, valloader: DataLoader, epochs: int) -> None:
    for epoch in range(epochs):
        for client_id, client_trainloader in trainloaders.items():
            # Train VAE on client data
            vae_train(vae, client_trainloader, epochs=10)

            # Share distribution information with global server
            distribution_info = get_distribution_info(vae)
            send_distribution_info(distribution_info)

            # Receive distribution information from other clients
            other_distribution_info = receive_distribution_info()

            # Generate augmented data using received distribution information
            augmented_data = generate_augmented_data(vae, other_distribution_info["normal"])

            # Train classification model using local, augmented, and validation data
            train(net, client_trainloader, valloader, epochs=10)

            # Send model updates to global server
            send_model_update(client_id, net.state_dict())

# Define logic to receive distribution information from global server
def receive_distribution_info() -> Dict:
    # Receive distribution information logic
    distribution_info = {

        "normal": {
            "mean": np.zeros(20),  # Adjust the size based on your latent space dimension
            "std": np.ones(20)
        }
    }
    return distribution_info

def send_model_update(client_id: str, model_update: Dict) -> None:
    # Implement the logic to send the model update to the global server
    # This can involve using a network protocol, a message queue, or any other communication mechanism
    # to send the model update

    # Example implementation:
    # Send the model update to the global server using a network protocol
    # For example, you can use the `socket` module to send the information over a network
    # or use a message queue like `RabbitMQ` to send the information
    pass

# Define global server procedure
def global_server() -> None:
    net = Net()
    x_dim = 3 * 32 * 32  # CIFAR-10 input size
    h_dim = 400
    z_dim = 20
    vae = VAE(x_dim, h_dim, z_dim)  # Initialize VAE object with required arguments

    # Initialize clients
    num_clients = 5  # Define the number of clients
    clients = initialize_clients(train_set, transform, batch_size=128, num_clients=num_clients)

    # Train model using FedDIS
    federated_train(net, vae, clients, trainloader, valloader, epochs=10)

    # Evaluate final model
    test_accuracy, tp, fp, tn, fn, precision, recall, f1 = evaluate(net, testloader)
    print(f"Test Accuracy: {test_accuracy:.2f}%")

    print("True Positives (TP):", tp)
    print("False Positives (FP):", fp)
    print("True Negatives (TN):", tn)
    print("False Negatives (FN):", fn)
    print(":", fn+tn+tp+fp)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

if __name__ == "__main__":
    global_server()

Files already downloaded and verified
Files already downloaded and verified
Random Images per Class: [5958 6008 5989 5987 5944 5912 6049 6086 5993 6074]
Epoch [1/10], Training Loss: 2.306, Validation Accuracy: 9.51%
Epoch [2/10], Training Loss: 2.305, Validation Accuracy: 9.51%
Epoch [3/10], Training Loss: 2.304, Validation Accuracy: 9.51%
Epoch [4/10], Training Loss: 2.303, Validation Accuracy: 9.59%
Epoch [5/10], Training Loss: 2.302, Validation Accuracy: 10.49%
Epoch [6/10], Training Loss: 2.301, Validation Accuracy: 12.03%
Epoch [7/10], Training Loss: 2.300, Validation Accuracy: 12.99%
Epoch [8/10], Training Loss: 2.299, Validation Accuracy: 13.97%
Epoch [9/10], Training Loss: 2.297, Validation Accuracy: 14.28%
Epoch [10/10], Training Loss: 2.296, Validation Accuracy: 14.70%
Epoch [1/10], Training Loss: 2.295, Validation Accuracy: 15.30%
Epoch [2/10], Training Loss: 2.293, Validation Accuracy: 16.12%
Epoch [3/10], Training Loss: 2.289, Validation Accuracy: 16.74%
Epoch [4/10], Trai

In [ ]:
# Define global server procedure
def global_server() -> None:
    net = Net()
    x_dim = 3 * 32 * 32  # CIFAR-10 input size
    h_dim = 400
    z_dim = 20
    vae = VAE(x_dim, h_dim, z_dim)  # Initialize VAE object with required arguments

    # Initialize clients
    num_clients = 5  # Define the number of clients
    clients = initialize_clients(train_set, transform, batch_size=128, num_clients=num_clients)

    # Train model using FedDIS
    federated_train(net, vae, clients, trainloader, valloader, epochs=10)

    # Evaluate final model
    test_accuracy, tp, fp, tn, fn, precision, recall, f1 = evaluate(net, testloader)
    print(f"Test Accuracy: {test_accuracy:.2f}%")

    print("True Positives (TP):", tp)
    print("False Positives (FP):", fp)
    print("True Negatives (TN):", tn)
    print("False Negatives (FN):", fn)
    print(":", fn+tn+tp+fp)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

if __name__ == "__main__":
    global_server()